In [1]:
import json
import os
import sys
import time
import platform
import pathlib

from DrissionPage import WebPage
from DrissionPage.common import Keys
from DrissionPage.common import By
from DrissionPage.common import wait_until
from DrissionPage.common import make_session_ele
from DrissionPage.common import configs_to_here

from urllib3.exceptions import InsecureRequestWarning

import warnings
warnings.simplefilter('ignore',InsecureRequestWarning)

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# ocr
import base64

try:
    import ddddocr
    from NonBrowser import NonBrowser
except Exception as exc:
    pass

import argparse

CONST_APP_VERSION = "Max Interpark Bot (2023.02.17)"

CONST_MAXBOT_CONFIG_FILE = 'settings.json'
CONST_MAXBOT_LAST_URL_FILE = "MAXBOT_LAST_URL.txt"
CONST_MAXBOT_INT28_FILE = "MAXBOT_INT28_IDLE.txt"

CONST_HOMEPAGE_DEFAULT = "https://www.globalinterpark.com/"
CONST_INTERPARK_SIGN_IN_URL = "https://www.globalinterpark.com/user/signin"

CONST_CHROME_VERSION_NOT_MATCH_EN="Please download the WebDriver version to match your browser version."
CONST_CHROME_VERSION_NOT_MATCH_TW="請下載與您瀏覽器相同版本的WebDriver版本，或更新您的瀏覽器版本。"

CONST_FROM_TOP_TO_BOTTOM = u"from top to bottom"
CONST_FROM_BOTTOM_TO_TOP = u"from bottom to top"
CONST_RANDOM = u"random"
CONST_SELECT_ORDER_DEFAULT = CONST_FROM_TOP_TO_BOTTOM

CONST_WEBDRIVER_TYPE_SELENIUM = "selenium"
CONST_WEBDRIVER_TYPE_UC = "undetected_chromedriver"
CONST_WEBDRIVER_TYPE_DRISSONPAGE = "drissonpage"

In [2]:
def get_app_root():
    # 讀取檔案裡的參數值
    basis = ""
    if hasattr(sys, 'frozen'):
        basis = sys.executable
    else:
        basis = sys.argv[0]
    app_root = os.path.dirname(basis)
    return app_root



def get_config_dict(args):
    app_root = get_app_root()
    
    config_filepath = os.path.join(app_root, CONST_MAXBOT_CONFIG_FILE)
    print("config file:", config_filepath)
    
    # allow assign config by command line.
    if not args.input is None:
        if len(args.input) > 0:
            config_filepath = args.input

    config_dict = None
    if os.path.isfile(config_filepath):
        with open(config_filepath) as json_data:
            config_dict = json.load(json_data)
    return config_dict

def write_last_url_to_file(url):
    outfile = None
    if platform.system() == 'Windows':
        outfile = open(CONST_MAXBOT_LAST_URL_FILE, 'w', encoding='UTF-8')
    else:
        outfile = open(CONST_MAXBOT_LAST_URL_FILE, 'w')

    if not outfile is None:
        outfile.write("%s" % url)


def read_last_url_from_file():
    ret = ""
    with open(CONST_MAXBOT_LAST_URL_FILE, "r") as text_file:
        ret = text_file.readline()
    return ret

def load_drissonpage(config_dict):
    driver = WebPage()
    return driver


def get_driver_by_config(config_dict):
    global driver

    # read config
    homepage = config_dict["homepage"]

    if config_dict is None:
        homepage = ""
    if len(homepage) == 0:
        homepage = CONST_HOMEPAGE_DEFAULT

    Root_Dir = get_app_root()
    webdriver_path = os.path.join(Root_Dir, "webdriver")
    print("platform.system().lower():", platform.system().lower())

    if config_dict["browser"] in ["chrome", "brave"]:
        # method 6: Selenium Stealth
        if config_dict["webdriver_type"] == CONST_WEBDRIVER_TYPE_SELENIUM:
            driver = load_chromdriver_normal(config_dict, config_dict["webdriver_type"])
        elif config_dict["webdriver_type"] == CONST_WEBDRIVER_TYPE_UC:
            # method 5: uc
            if platform.system().lower() == "windows":
                if hasattr(sys, 'frozen'):
                    from multiprocessing import freeze_support
                    freeze_support()
            driver = load_chromdriver_uc(config_dict)
        elif config_dict["webdriver_type"] == CONST_WEBDRIVER_TYPE_DRISSONPAGE:
            # method DrissonPage
            if platform.system().lower() == "windows":
                if hasattr(sys, 'frozen'):
                    from multiprocessing import freeze_support
                    freeze_support()
            driver = load_drissonpage(config_dict)

    if driver is None:
        print("create web driver object fail @_@;")
    else:
        try:
            print("goto url:", homepage)

            if 'globalinterpark.com' in homepage:
                if len(config_dict["advanced"]["interpark_account"])>0:
                    homepage = CONST_INTERPARK_SIGN_IN_URL

            driver.get(homepage)
            time.sleep(3.0)
        except Exception as exce1:
            print('get URL Exception:', exce1)
            pass

    return driver


def get_current_url(driver):
    DISCONNECTED_MSG = ': target window already closed'

    url = ""
    is_quit_bot = False

    try:
        url = driver.url
    except Exception as exc:
        logger.error('Maxbot URL Exception')
        logger.error(exc, exc_info=True)

        # UnicodeEncodeError: 'ascii' codec can't encode characters in position 63-72: ordinal not in range(128)
        str_exc = ""
        try:
            str_exc = str(exc)
        except Exception as exc2:
            pass

        if len(str_exc) == 0:
            str_exc = repr(exc)

        exit_bot_error_strings = ['Max retries exceeded'
        , 'chrome not reachable'
        , 'unable to connect to renderer'
        , 'failed to check if window was closed'
        , 'Failed to establish a new connection'
        , 'Connection refused'
        , 'disconnected'
        , 'without establishing a connection'
        , 'web view not found'
        , 'invalid session id'
        ]
        for each_error_string in exit_bot_error_strings:
            if isinstance(str_exc, str):
                if each_error_string in str_exc:
                    print('quit bot by error:', each_error_string)
                    is_quit_bot = True
                    driver.quit()
                    sys.exit()

        # not is above case, print exception.
        print("Exception:", str_exc)
        pass

    return url, is_quit_bot

In [3]:
class Args:
    def __init__(self, input = None):
        self.input = input

args = Args(input="/Users/yazheng/code/learn/interpark_bot/settings.json")

config_dict = get_config_dict(args)

driver = None
if config_dict is not None:
    driver = get_driver_by_config(config_dict)
else:
    print("Load config error!")

# internal variable. 說明：這是一個內部變數，請略過。
url = ""
last_url = ""

interpark_dict = {}
interpark_dict["opener_popuped"] = False
interpark_dict["main_window_handle"] = None
interpark_dict["is_step_1_submited"] = False

ocr = None
try:
    if config_dict["ocr_captcha"]["enable"]:
        ocr = ddddocr.DdddOcr(show_ad=False, beta=config_dict["ocr_captcha"]["beta"])
except Exception as exc:
    print(exc)
    pass

config file: /opt/homebrew/Caskroom/miniforge/base/envs/crawler/lib/python3.10/site-packages/settings.json
platform.system().lower(): darwin
goto url: https://www.globalinterpark.com/


In [30]:
if driver is None:
    print("web driver not accessible!")

url, is_quit_bot = get_current_url(driver)
is_maxbot_paused = False
if os.path.exists(CONST_MAXBOT_INT28_FILE):
    is_maxbot_paused = True

if len(url) > 0:
    if url != last_url:
        print(url)
        write_last_url_to_file(url)
        if is_maxbot_paused:
            print("MAXBOT Paused.")
    last_url = url

if is_maxbot_paused:
    time.sleep(0.2)

https://www.globalinterpark.com/login?lang=en


In [32]:
if "globalinterpark.com" in url:
    if interpark_dict["main_window_handle"] is None:
        interpark_dict["main_window_handle"] = driver.get_tab()

In [35]:
def escape_to_first_tab(driver, main_window_handle):
    try:
        tabs = driver.tabs()
        window_handles_count = len(tabs)
        if window_handles_count == 1:
            pass
        if window_handles_count > 1:
            for tab_id in tabs:
                # switch focus to child window
                if (tab_id != main_window_handle):
                    # driver.switch_to.window(w)
                    driver.set.activate(tab_id)
                    break
    except Exception as e:       
        print("Escape tab exception: ", e)


In [ ]:
escape_to_first_tab(driver, interpark_dict["main_window_handle"])

In [34]:
def interpark_main(driver, config_dict, url, ocr, interpark_dict):
    escape_to_first_tab(driver, interpark_dict["main_window_handle"])
    
interpark_dict = interpark_main(driver, config_dict, url, ocr, interpark_dict)

'https://www.globalinterpark.com/login?lang=en'